# All water supply systems (clustering)

CEDAE is the coorporation that provides drinking water and wastewater services for the Rio de Janeiro State.
They provide plenty of data regarding the quality of the water for the press and for the population, due to laws imposed by the Ministry of Health of Brazil.

Here we aim to extract, compile and analyse data from all water supply systems (that we'll be denoting by the acronym *WSS*) being monitored by CEDAE. This data is also available on their webpage. The data of every WSS is routinely gathered and anually compiled into a single report.

Unfortunately, this data is not presented in high granularity, as only the mean of the measurements per month are available. Although this data has been monitored and available since 2004, we have less data samples (\~204) than the number of daily recorded samples in a single year (\~365).

The data available contains the following parameters:

* Physical and Chemical
  * Haze (*turbidez*)
  * Aparent color
  * Residual chlorine
* Bacteriological
  * Total coliforms (before and after recollection)
  * E. coli (before and after recollection)

The plan of this notebook is to do the following sequence of tasks:

1. Download the HTML page where all links to PDFs reside
2. Parse the HTML page and extract any link to PDFs and its metadata (year and WSS)
3. Cluster similarly-named WSSs and download their PDFs in the input/ folder
4. Cluster old reports by inspecting their PDFs and moving/renaming them to the input/ folder too

## 1. Download the HTML page

First we download the HTML using the `urllib.request.urlopen` method. It returns a file pointer, from which the page can be read as a stream of bytes, and decoded to UTF-8, the default string encoding for Python (and the modern internet, pretty much).

In [1]:
import urllib.request

page_url = 'https://cedae.com.br/relatorioanual'
with urllib.request.urlopen(page_url) as fp:
    page = fp.read().decode() # Read from page and decode to UTF-8 string

You can check that we got indeed the HTML for the page:

In [2]:
print(page[:80])

<!DOCTYPE html>
<html  lang="pt-BR">
<head id="Head">
<!--*******************


## 2. Parse the HTML page

For this task, we'll be using the `HTMLParser` class from the `html.parser` module, which allows us to specify callbacks for when the parser reads the beggining tags (`<...>`), in-between text (`<a> ... </a>`), and their ending tags (`</...>`).

After analysing the source code for the page we're parsing, we notice that all links that interest us are contained in tables, particularly inside `<td>` tags. Moreover, every table has a top row whose class is `thead` (probably for short for "table head") containing the year of the reports.

For building our custom parser, we inherit the `HTMLParser`.

In [3]:
import html.parser
from datetime import datetime

class MyHTMLParser(html.parser.HTMLParser):
    
    def __init__(self):
        super().__init__()
        self.in_td = False
        self.in_thead = False
        self.year = None
        self.link = None
        self.links = {}
    
    def handle_starttag(self, tag, attrs):
        if tag == 'td':
            self.in_td = True
        elif tag == 'a':
            links = [v for k, v in attrs if k == 'href']
            if links:
                assert len(links) == 1, links
                self.link = links[0]
        elif tag == 'tr' and ('class', 'thead') in attrs:
            self.in_thead = True
    
    def handle_data(self, data):
        if self.in_thead:
            for number in [int(s) for s in data.split() if s.isdigit()]:
                # CEDAE was created in 1975
                if 1975 <= number <= datetime.now().year:
                    self.year = number
        elif self.in_td and self.link is not None:
            assert self.year is not None
            data = data.strip()
            assert len(data) > 0
            if self.year not in self.links:
                self.links[self.year] = {}
            assert data not in self.links[self.year], self.links[self.year][data]
            self.links[self.year][data] = self.link
    
    def handle_endtag(self, tag):
        if tag == 'td':
            self.in_td = False
        elif tag == 'a':
            self.link = None
        elif tag == 'tr':
            self.in_thead = False
        elif tag == 'table':
            self.year = None

We now construct a parser instance and feed it with the contents of the HTML page.

The links are stored in the `links` field. It is a dictionary of dictionaries of strings. It is first indexed by the year of the reports, and second by the name of the water supply system, resulting in the link to the PDF of the annual report corresponding to that WSS in that year.

In [4]:
parser = MyHTMLParser()
parser.feed(page)
links = parser.links

## 3. Cluster similarly-named WSSs and download their PDFs in the input/ folder

We know that some water supply systems are wrongly named. Let's create a list of all unique names.

In [5]:
import numpy as np
names = set()
for year, reports in links.items():
    names |= reports.keys()
names = np.asarray(sorted(names))

Then, we create an index that relates names to indices in this list.

In [6]:
name_index = {v: i for i, v in enumerate(names)}

Now we create a list whose item `a[i]` contains the years of reports of the `i-th` string in the list.

In [7]:
name_reports = [{year for year, reports in links.items() if name in reports} for name in names]

Let's cluster names that don't have reports in the same year, but are very similar according to the Levenshtein distance.

In [8]:
import numpy as np
from scipy.sparse import csr_matrix
import Levenshtein

nnames = len(names)
dmatrix = np.zeros(shape=(nnames, nnames), dtype=int)  # distance matrix

# Distances where user input will be requested
# to evaluate whether two names belong in the same cluster
min_dist = 4
max_dist = 5

for i in range(nnames):
    for j in range(i):
        dist = Levenshtein.distance(names[i], names[j])
        # We evaluate first names that are substring
        if names[i] in names[j] or names[j] in names[i]:
            dist = min_dist
        dmatrix[i][j] = dmatrix[j][i] = dist

# matrix indices, sorted by distance (lower first)
dmatrix_indices = sorted(((i, j) for i in range(nnames) for j in range(i)), key=lambda k: dmatrix[k[0]][k[1]])

min_year = min(links.keys())
max_year = max(links.keys())
nyears = max_year - min_year + 1

yearmap = np.zeros(shape=(nnames, nyears), dtype=bool)  # year map

for i in range(nnames):
    for year in name_reports[i]:
        yearmap[i][year - min_year] = True

clusters = np.arange(nnames)  # cluster ids (initially every name is in its own cluster)

not_connected = np.zeros(shape=(nnames, nnames), dtype=bool)  # from input

for i, j in dmatrix_indices:
    dist = dmatrix[i][j]
    if dist > max_dist:
        break
    ci = clusters[i]
    cj = clusters[j]
    yi = np.any(yearmap[clusters == ci], axis=0)
    yj = np.any(yearmap[clusters == cj], axis=0)
    if np.any(yi & yj):
        continue  # cannot merge clusters
    if np.any(not_connected[clusters == ci, clusters == cj]):
        continue  # not merged before
    ci_str = ", ".join(names[clusters == ci])
    cj_str = ", ".join(names[clusters == cj])
    if dist >= min_dist:
        ok = input("({}) = ({})? ([Y]es/[n]o/e[x]it) ".format(ci_str, cj_str))
        if 'x' in ok:
            break
        elif 'n' in ok:
            not_connected[i][j] = not_connected[j][i] = 1
            continue  # doesn't want to merge clusters
    clusters[clusters == cj] = ci

(Bom Jesus do Itabapoana) = (Bom Jesus)? ([Y]es/[n]o/e[x]it) 
(Cordeiro/Cantagado) = (Cordeiro)? ([Y]es/[n]o/e[x]it) 
(Coronel Teixeira/Batatal, Coronel Teixeirabatatal) = (Coronel Teixeira)? ([Y]es/[n]o/e[x]it) 
(Laranjal) = (Imunana Laranjal)? ([Y]es/[n]o/e[x]it) 
(Nossa Senhora Aparecida) = (Aparecida)? ([Y]es/[n]o/e[x]it) 
(Ourania) = (Cacaria)? ([Y]es/[n]o/e[x]it) n
(Palmas Paulo De Frontin) = (Palmas)? ([Y]es/[n]o/e[x]it) 
(Pipeiras/Palacete) = (Pipeiras)? ([Y]es/[n]o/e[x]it) 
(São Sebastião dos Ferreiros) = (Ferreiros)? ([Y]es/[n]o/e[x]it) 
(Trajano de Morais) = (Trajano)? ([Y]es/[n]o/e[x]it) 
(Ourania) = (Osório)? ([Y]es/[n]o/e[x]it) n


Now, we need to choose a representable sample from each cluster. We'll define a list of criterium for getting the "best" name. We also create a file for storing the "codenames" and real names of each water supply system.

In [9]:
import unidecode
import pandas as pd
import os

def non_ascii_char_count(s):
    return len([c for c in s if not c.isascii()])

def upper_case_char_count(s):
    return len([c for c in s if not c.isupper()])

def year_of_last_report(name):
    index = name_index[name]
    reports = name_reports[index]
    return max(reports)

def choose_name(cluster):
    cluster = sorted(cluster, key=year_of_last_report)
    cluster = sorted(cluster, key=non_ascii_char_count)
    cluster = sorted(cluster, key=upper_case_char_count)
    cluster = sorted(cluster, key=len)
    return cluster[-1]

def convert_char(c):
    if c.isalpha():
        return c.lower()
    else:
        return '_'

def format_name(name):
    name = unidecode.unidecode(name)
    g = map(convert_char, name)
    return ''.join(list(g))

try:
    os.mkdir('output')
except FileExistsError:
    pass  # it's ok if the output folder already exists

names_file = os.path.join('output', 'wss_codenames.csv')
names_df = pd.DataFrame(columns=('name',))

cluster_names = {}
for ci in np.unique(clusters):
    ci_names = names[clusters == ci]
    name = choose_name(ci_names)
    codename = format_name(name)
    cluster_names[ci] = codename
    names_df.loc[codename] = (name,)

names_df.to_csv(names_file)

Now, let's download the PDFs and store them in a nice hierarchichal directory structure composed of `input/<year>/<wss>.pdf`.

In [10]:
import os
import urllib.parse

page_url_parts = urllib.parse.urlparse(page_url)
base_url = page_url_parts._replace(path='').geturl()

def normalize_url(url):
    if url.startswith('/'):
        url = base_url + url
    # Heuristic: if URL has %, it must be already formatted.
    # If not, format only the URL path
    if '%' in url:
        return url
    else:
        urlparts = url.split('/')
        filename = urllib.parse.quote(urlparts[-1])
        return '/'.join(urlparts[:-1] + [filename])

base_path = 'input'
for year, reports in links.items():
    year_path = os.path.join(base_path, str(year))
    try:
        os.mkdir(year_path)
    except FileExistsError:
        pass  # it's ok if such folder already exists
    for name, report_url in reports.items():
        index = name_index[name]
        cluster = clusters[index]
        cname = cluster_names[cluster]
        filename = cname + ".pdf"
        filepath = os.path.join(year_path, filename)
        if os.path.exists(filepath) and os.path.isfile(filepath):
            continue  # don't need to download files already local
        report_url = normalize_url(report_url)
        with urllib.request.urlopen(report_url) as webfp, open(filepath, 'wb') as localfp:
            localfp.write(webfp.read())

### 4. Cluster old reports by inspecting their PDFs and moving/renaming them to the input/ folder too

Old reports are zipped and available for download in the same page as the more recent reports. For simplicity, we have already downloaded them in the `input/old` folder.

In this section, we are going to inspect the textual contents of these PDFs and guess which WSS are they related to.

First, let's list every report located in the `input/old` folder.

In [35]:
import os
import glob

old_reports = {}
for year in os.listdir(os.path.join('input', 'old')):
    pathname = os.path.join('input', 'old', year, '*.pdf')
    assert year.isdigit(), "Assumed folders are numbers"
    old_reports[int(year)] = list(glob.iglob(pathname))

Now, for each year, we'll define regular expression patterns for obtaining the name.

In [106]:
def get_regex_specs(year):
    if year < 2009:
        return None
#         return [("([Nn]?[Oo]s? )?[Mm]unicípios? d[eo] ([^,.]*?),? "\
#                  "(é|são|recebe|na região|e o distrito|somente o distrito|"\
#                  "a CEDAE|no município do Rio de Janeiro)", 2)]
    elif year == 2009:
#        return [(r"SOBRE O SISTEMA (DE )?(.*?)\s*O MANANCIAL", 2)]
        return None
    elif year == 2010:
        return None
#        return [(r"SOBRE O SISTEMA (DE )?(.*?)\s*O MANANCIAL", 2)]
    elif year == 2011:
        return None
#        return [(r"SOBRE O SISTEMA (DE )?(.*?)\s*o O MANANCIAL", 2)]
    else:
        return None

In [107]:
import PyPDF2
import re

eol_hyphen_patt = re.compile('-\n')
conj_article_patt = re.compile(' [Ee]( [OAoa])? ')
spaces_patt = re.compile(r'\s\s+')
old_names = {}
for year, reports in old_reports.items():
    yearlytexts = {}
    regex_spec = get_regex_specs(year)
    if regex_spec is None:
        continue
    old_names[year] = {}
    patt_strs, groups = zip(*regex_spec)
    patts = list(map(re.compile, patt_strs))
    for report in reports:
        reporttext = ""
        reader = PyPDF2.PdfFileReader(report)
        for page in reader.pages:
            text = page.extractText()
            text = eol_hyphen_patt.sub('', text)
            text = spaces_patt.sub(r' ', text)
            reporttext += text
        names = set()
        for patt, group in zip(patts, groups):
            for match in patt.finditer(reporttext):
                name = match[group]
                name = conj_article_patt.sub('/', name)
                names.add(name)
        if names:
            name = '/'.join(sorted(names))
        else:
            basename = os.path.basename(report)
            name, ext = os.path.splitext(basename)
            name = name.replace('_', ' ')
            name = conj_article_patt.sub('/', name)
        old_names[year][report] = name.title()

In [108]:
for path, name in old_names[2011].items():
    print("{:s} ({:s})".format(name, path.split('/')[-1]))

KeyError: 2011